<a href="https://colab.research.google.com/github/taresadere/taresadere/blob/main/freeze_inception_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install categorical


  Created wheel for categorical: filename=categorical-0.1.4-py3-none-any.whl size=4552 sha256=f36a50bdef90096875a72f0cf004d4cfd8267885793d5a8563f504ce9c3da13b
  Stored in directory: /root/.cache/pip/wheels/ea/0c/fd/9c5ade13a589a48840b29238ab565144bccd1eee7c276795d3
Successfully built categorical


In [ ]:
!pip install to_categorical

ERROR: Could not find a version that satisfies the requirement to_categorical (from versions: none)
ERROR: No matching distribution found for to_categorical


In [ ]:
import tensorflow as tf
import pandas as pd 
import random
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, array_to_img, img_to_array
from tensorflow.keras.models import Sequential
from glob import glob

In [ ]:

train_path = "drive/My Drive/train"
valid_path ="drive/My Drive/validation"
test_path = "drive/My Drive/test"
IMAGE_SIZE=(224, 224)
from keras.preprocessing import image
classes= ["normal", "follicular", "intense", "scar", "trichiasis", "corneal opacity"]
def model_predict(img_path, model):
    image = cv2.imread(img_path)
    #image = cv2.resize(image, (224, 224))
    image = cv2.resize(image,   IMAGE_SIZE)
    image = image.astype("float") / 255.0
    #image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    res = model.predict(image)
    label = np.argmax(res)
    print("Label", label)
    labelName = classes[label]
    print("Label name:", labelName)
    return labelName


In [ ]:
folders = glob("drive/My Drive//train/*")
folders

['drive/My Drive//train/intense',
 'drive/My Drive//train/follicular',
 'drive/My Drive//train/corneal',
 'drive/My Drive//train/normal',
 'drive/My Drive//train/trichiasis',
 'drive/My Drive//train/scar']

In [ ]:
inception = InceptionV3(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

87924736/87910968 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras import Model 
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(0.2)(x)
output  = Dense(units=6, activation='softmax')(x)
model = Model(inception.input, output)
for layer in vggModel.layers:
    layer.trainable = False

In [ ]:
# Create a model object
#model = Model(inputs=inception.input, output=prediction)

# View the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=loss, metrics= ['accuracy'])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
valid_datagen =ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory("drive/My Drive/train",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 179 images belonging to 6 classes.


In [ ]:
valid_set = valid_datagen.flow_from_directory("drive/My Drive/validation",
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 76 images belonging to 6 classes.


In [ ]:
# fit the model, log the results and the training time
now = datetime.datetime.now
t = now()
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
lrr = ReduceLROnPlateau(monitor='val_accuracy', 
                        patience=3, 
                        verbose=1, 
                        factor=0.4, 
                        min_lr=0.0001)


callbacks = [lrr]
# fit the model, it will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=valid_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(valid_set)
)
print('Training time: %s' % (now() - t))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
6/6 [==============================] - 77s 13s/step - loss: 1.7316 - accuracy: 0.2626 - val_loss: 2.3322 - val_accuracy: 0.1711
Epoch 2/5
6/6 [==============================] - 44s 8s/step - loss: 1.4254 - accuracy: 0.4804 - val_loss: 2.4283 - val_accuracy: 0.2368
Epoch 3/5
6/6 [==============================] - 44s 8s/step - loss: 1.1785 - accuracy: 0.6089 - val_loss: 2.2978 - val_accuracy: 0.2368
Epoch 4/5
6/6 [==============================] - 47s 8s/step - loss: 0.9609 - accuracy: 0.7095 - val_loss: 2.5078 - val_accuracy: 0.2500
Epoch 5/5
6/6 [==============================] - 45s 8s/step - loss: 0.8108 - accuracy: 0.7374 - val_loss: 2.6007 - val_accuracy: 0.2368
Training time: 0:05:02.844630


In [ ]:
score = model.evaluate_generator( valid_set)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


Test Score: 3.0827372074127197
Test Accuracy: 0.25


In [ ]:
model.save("C:/Users/Dare/Desktop/WEB APP/models/inceptionv3.h5")

In [ ]:
model.save_weights('inceptionv3.hdf5')

In [ ]:
from keras.models import load_model
model = load_model("drive/My Drive/inceptionv3.h5")

# \\::# making prediction and classification report
```


In [ ]:
batch_size=32
y_pred = model.predict(valid_set, batch_size=batch_size)

In [ ]:
from sklearn.metrics import classification_report
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix,roc_curve,auc
from PIL import Image
y_pred=model.predict_generator(generator=valid_set,)
y_pred=np.argmax(y_pred,axis=1)
print('classification report')
print(classification_report(valid_set.classes, y_pred))

# plot confusion matrix 

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
def plot_confusion_matrix(normalize):
  classes = ['normal','follicular', 'intense', 'scar', 'trichiasis','corneal opacity']
  tick_marks = [0.5,1.5]
  cn = confusion_matrix(valid_set, y_pred, normalize=normalize)
  sns.heatmap(cn,cmap='plasma',annot=True)
  plt.xticks(tick_marks, classes)
  plt.yticks(tick_marks, classes)
  plt.title('Confusion Matrix')
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

print('Confusion Matrix without Normalization')
plot_confusion_matrix(normalize=None)

print('Confusion Matrix with Normalized Values')
plot_confusion_matrix(normalize='true')

Confusion Matrix without Normalization


ValueError: ignored

# visualization of accuracy/loss

In [ ]:
N= 100
plt.style.use("ggplot")
plt.figure()
plt.plot(r.history["loss"], label="train loss")
plt.plot( r.history["val_loss"], label="val_loss")
plt.plot(r.history["accuracy"], label="train accuracy")
plt.plot( r.history["val_accuracy"], label="val_accuracy")
plt.title("training loss and Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("drive/My Drive/inception")